In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import re

In [ ]:
def sql_connection():
    try:
        con = sqlite3.connect('FlightDatabase.db')
        print('Connected to: FlightDatabase.db' )
        return con
    except Error:
        print(Error)
        
def create_table(con, create_sql):
    cursorObj = con.cursor()
    cursorObj.execute(create_sql)
    con.commit()            
    
def list_tables(con):
    cursorObj = con.cursor()
    cursorObj.execute('SELECT name from sqlite_master where type= "table"')
    print(cursorObj.fetchall())     

def sql_insert(con, insert_sql):
    cursorObj = con.cursor()    
    cursorObj.execute(insert_sql)
    con.commit()  
    
def table_schema(con, table):
    cursorObj = con.cursor()    
    print(sql_fetch(con, f"PRAGMA table_info('{table}');"))           
    
def sql_fetch(con, select_sql):
    cursorObj = con.cursor()
    cursorObj.execute(select_sql)
    rows = cursorObj.fetchall()
    for row in rows:
        print(row) 
        
def is_valid(row):
    col_is_3bigletter = [1,2,3,4]
    col_is_datetime = [5,6] 
    is_correct = True 
    #print(row)
    if len(row)== 7:         
        for i in col_is_3bigletter:
            if not re.match('^[A-Z]{3}$', row[i]):
                is_correct = False
        for i in col_is_datetime:
            if not re.match('^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$', row[i]):
                is_correct = False    
    else:
        is_correct = False 
            
    return is_correct    

# 1.     Korzystając z biblioteki sqlite3 utwórz nową bazę danych z tabelą FlightLeg, która będzie zawierała informację o lotach samolotów, z następującymi kolumnami: 

·       id — identyfikator numeryczny nadawany z sekwencji 

·       tailNumber — identyfikator samolotu 

·       sourceAirportCode, destinationAirportCode — trzyliterowy kod lotniska (wg IATA) 

·       sourceCountryCode, destinationCountryCode — trzyliterowy kod kraju (wg ISO 3166-1 Alpha-3) 

·       departureTimeUtc, landingTimeUtc — data i czas (z dokładnością do sekundy) odpowiednio odlotu i lądowania samolotu (w UTC) 

In [ ]:
create_sql ="""
CREATE TABLE if not exists FlightLeg(
id INTEGER PRIMARY KEY AUTOINCREMENT, 
tailNumber CHAR(10), 
sourceAirportCode CHAR(3), 
destinationAirportCode CHAR(3), 
sourceCountryCode CHAR(3), 
destinationCountryCode CHAR(3),
departureTimeUtc DATETIME,
landingTimeUtc DATETIME
)
"""
con = sql_connection()
create_table(con, create_sql)
con.close()

In [ ]:
con = sql_connection()
list_tables(con)
table_schema(con, 'FlightLeg')
con.close()

# 2.     Wypełnij tabelę FlightLeg danymi z pliku: https://bitpeak.pl/datasets/flightlegs.csv 

In [ ]:
df = pd.read_csv('flightlegs.csv', delimiter=';')
#df = pd.read_csv('test.csv', delimiter=';')
df = df.where(df.notnull(), 'None')
con = sql_connection()
for x in range(0,len(df.index)):    
    row = df.loc[x, :].values.tolist()
    if is_valid(row):
        entities = f"(NULL, {str(row)[1:-1]})"
        insert_sql = f"INSERT INTO FlightLeg VALUES{entities}"
        #print(insert_sql)
        sql_insert(con, insert_sql)  
con.close()

In [ ]:
con = sql_connection()
select_sql= """
SELECT  * 
--count(*)
FROM FlightLeg
WHERE 1=1
LIMIT 10;
"""
sql_fetch(con, select_sql)
con.close()

# 3.     Dodaj do tabeli dwie nowe kolumny: 

·       flightDuration — wypełnij ją wartościami określającymi czas trwania lotu w minutach (zaokrągl wartości do najbliższej liczby całkowitej) 

In [ ]:
con = sql_connection()
alter_sql="ALTER TABLE FlightLeg ADD COLUMN flightDuration INTEGER"
cursorObj = con.cursor()
cursorObj.execute(alter_sql)
con.commit()
print(sql_fetch(con, select_sql))
con.close()

In [ ]:
con = sql_connection()
update_sql= """
UPDATE FlightLeg
SET flightDuration = Cast (( JulianDay(landingTimeUtc) - JulianDay(departureTimeUtc)) * 24 * 60 As Integer)
"""
cursorObj = con.cursor()
cursorObj.execute(update_sql)
con.commit()
print(sql_fetch(con, select_sql))
con.close()

·       flightType — wypełnij ją wartościami określającymi typ lotu: krajowy (wartość ‘D’ = domestic) lub zagraniczny (wartość ‘I’ = international); lot krajowy to lot zaczynający się i kończący w tym samym kraju 

In [ ]:
con = sql_connection()
alter_sql="ALTER TABLE FlightLeg ADD COLUMN flightType CHAR(1)"
cursorObj = con.cursor()
cursorObj.execute(alter_sql)
con.commit()
print(sql_fetch(con, select_sql))
con.close()

In [ ]:
con = sql_connection()
update_sql= """
UPDATE FlightLeg
SET flightType = Case when sourceCountryCode == destinationCountryCode then 'D' else 'I' END
"""
cursorObj = con.cursor()
cursorObj.execute(update_sql)
con.commit()

print(sql_fetch(con, select_sql))
con.close()

# 4.     Zaimplementuj logikę odpowiadającą na poniższe pytania:



    1.     Który samolot wykonał najwięcej lotów? 

In [ ]:
select_sql= """
SELECT tailNumber, count(tailNumber) as flight_cnt
FROM FlightLeg
WHERE 1=1
group by tailNumber
ORDER BY flight_cnt DESC LIMIT 1;
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

    2.     Który samolot przeleciał najwięcej minut? 

In [ ]:
select_sql= """
SELECT tailNumber, SUM(flightDuration) AS sum_duration
FROM FlightLeg
WHERE 1=1
group by tailNumber
ORDER BY sum_duration DESC LIMIT 1;
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

    3.     Który lot, w podziale na krajowe i zagraniczne, był najkrótszy, a który najdłuższy, i ile minut trwał? 

In [ ]:
select_sql= """
SELECT 
id,
flightType,
flightDuration,
MAX(flightDuration)as MAX,
MIN(flightDuration)as MIN
FROM FlightLeg
where flightType ='I'
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

In [ ]:
select_sql= """
SELECT * FROM (
SELECT 
id,
flightType,
flightDuration,
'MAX' as type,
ROW_NUMBER() OVER(PARTITION BY flightType,flightType ORDER BY flightDuration DESC) rowNumber
FROM FlightLeg
) tmax
where tmax.rowNumber = 1

UNION

SELECT tmin.* FROM (
SELECT 
id,
flightType,
flightDuration,
'MIN' as type,
ROW_NUMBER() OVER(PARTITION BY flightType,flightType ORDER BY flightDuration ASC) rowNumber
FROM FlightLeg
)tmin
where tmin.rowNumber = 1
;
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

    4.     (punkt opcjonalny) Ile jest błędnych rekordów opisujących loty, które wskazują, że samolot wykonywał więcej niż jeden lot jednocześnie? Wyświetl wszystkie pary takich konfliktowych lotów.


In [ ]:
select_sql= """
SELECT 
FL1.*,
'||',
FL2.*
FROM FlightLeg AS FL1
JOIN FlightLeg AS FL2
on FL1.tailNumber = FL2.tailNumber
AND FL1.id <> FL2.id
AND(FL2.departureTimeUtc <= FL1.departureTimeUtc AND FL1.departureTimeUtc <= FL2.landingTimeUtc)
;
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

    5.     (punkt opcjonalny) Odstęp pomiędzy którymi lotami tego samego dowolnego samolotu był najkrótszy i ile minut trwał? 